<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/test_disitll.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

frankmorales@Franks-MacBook-Air ~ % ssh -i  /Users/frankmorales/lambda/poc-ft.pem ubuntu@192.222.53.76                                         
Welcome to Ubuntu 22.04.5 LTS (GNU/Linux 6.8.0-60-generic x86_64)
 .============.
 ||   __      ||    _                    _         _
 ||   \_\     ||   | |    __ _ _ __ ___ | |__   __| | __ _
 ||    \_\    ||   | |   / _` | '_ ` _ \| '_ \ / _` |/ _` |
 ||   /_λ_\   ||   | |__| (_| | | | | | | |_) | (_| | (_| |
 ||  /_/ \_\  ||   |_____\__,_|_| |_| |_|_.__/ \__,_|\__,_|
  .============.                                  GPU CLOUD

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/pro

Last login: Sat Sep 13 09:39:32 2025 from 107.171.187.251
To run a command as administrator (user "root"), use "sudo <command>".
See "man sudo_root" for details.

ubuntu@192-222-53-76:~$ . mistral/bin/activate
(mistral) ubuntu@192-222-53-76:~$  python -c "import torch; torch.cuda.empty_cache(); import gc; gc.collect()"
(mistral) ubuntu@192-222-53-76:~$  python qwen3tomistral-distill-V2.py
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
Successfully logged in to Hugging Face Hub.
Loading teacher model: Qwen/Qwen3-Next-80B-A3B-Instruct with quantization...
The fast path is not available because one of the required library is not installed. Falling back to torch implementation. To install follow https://github.com/fla-org/flash-linear-attention#installation and https://github.com/Dao-AILab/causal-conv1d
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [01:35<00:00,  2.34s/it]
Generating distillation dataset with teacher model...
Generating batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [09:39<00:00, 115.96s/it]
Dataset with 20 samples created.
Cleaning up teacher model to free memory...
Loading student model: mistralai/Mistral-7B-v0.1...
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.47s/it]
Setting up the SFTTrainer...
Generating train split: 3 examples [00:00, 640.51 examples/s]
Executing fine-tuning for 3 epochs...
Starting fine-tuning...
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
  0%|                                                                                                                                                                           | 0/3 [00:00<?, ?it/s]Casting fp32 inputs back to torch.float16 for flash-attn compatibility.
{'loss': 0.9107, 'grad_norm': 123838.6171875, 'learning_rate': 0.0, 'epoch': 1.0, 'num_input_tokens_seen': 0, 'train_runtime': 1.4158, 'train_tokens_per_second': 0.0}                                
{'loss': 0.9057, 'grad_norm': 115116.4921875, 'learning_rate': 4e-05, 'epoch': 2.0, 'num_input_tokens_seen': 0, 'train_runtime': 2.3569, 'train_tokens_per_second': 0.0}                              
{'loss': 0.7662, 'grad_norm': 90711.140625, 'learning_rate': 8e-05, 'epoch': 3.0, 'num_input_tokens_seen': 0, 'train_runtime': 3.3034, 'train_tokens_per_second': 0.0}                                
{'train_runtime': 3.8236, 'train_samples_per_second': 2.354, 'train_steps_per_second': 0.785, 'train_loss': 0.8608601093292236, 'epoch': 3.0, 'num_input_tokens_seen': 0}                             
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.27s/it]
Saving the fine-tuned model locally to ./mistral-7b-qwen-Next-80B-A3B-Instruct-distilled...
Uploading model to Hugging Face Hub: frankmorales2020/mistral-7b-qwen-Next-80B-A3B-Instruct-distilled...
Processing Files (1 / 1)                : 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|  168MB /  168MB, 64.6MB/s  
New Data Upload                         : 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|  168MB /  168MB, 64.6MB/s  
  ...distilled/adapter_model.safetensors: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|  168MB /  168MB            
No files have been modified since last commit. Skipping to prevent empty commit.
Model and model card successfully uploaded to frankmorales2020/mistral-7b-qwen-Next-80B-A3B-Instruct-distilled!
Distillation, fine-tuning, and upload complete!

In [ ]:
!pip install flash-attn --no-build-isolation -q

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# Load base + adapter (uses FP16 for efficiency)
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    dtype=torch.float16,
    device_map="auto",
    attn_implementation="flash_attention_2"  # Optional: For speed
)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(base_model, "frankmorales2020/mistral-7b-qwen-Next-80B-A3B-Instruct-distilled")

# Test prompt (from your dataset)
prompt = "### Instruction:\nWrite a Python function to compute Fibonacci numbers efficiently.\n\n### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

In [2]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Instruction:
Write a Python function to compute Fibonacci numbers efficiently.

### Response:

```python
def fibonacci(n):
    if n == 0 or n == 1:
        return n
    prev = 0
    curr = 1
    for i in range(2, n + 1):
        temp = curr
        curr += prev
        prev = temp
    return curr
```

### Explanation:

The function `fibonacci(n)` takes a single integer `n` as input and returns the `n`th Fibonacci number.

The function uses the recursive definition of the Fibonacci sequence, which states that the `n`th Fibonacci number is the sum of the `(n - 1)`th and `(n - 2)`th Fibonacci numbers.

The function uses the base case of `n == 0` or `n == 1` to return the input directly, as these are the initial values of the Fibonacci sequence.

The function then initializes two variables, `prev` and `curr`, to hold the previous and current Fibonacci numbers, respectively.

The function then uses a `for` loop to iterate from `2` to `n + 1`, computing the `n`th Fibonacci number by addi